In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pickle

from skimage.transform import resize
from skimage.io import imread
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

In [2]:
#Formatage des données
#Cette partie nécéssite beacoup de RAM

t1 = time.time()
    
Categories=['healthy','rot','rust'] #les différentes catégories qu'on traite

flat_data_arr = [] #contient les images aplaties
target_arr = [] #contient la sortie

datadir='New Plant Diseases Dataset(Augmented)\\New Plant Diseases Dataset(Augmented)\\train\\Apples'

N_img_each = 1250 #nombre d'images par catégorie

for i in Categories:
    
    print(f'loading... categorie : {i}')    
    path=os.path.join(datadir,i)    
    
    for img in os.listdir(path)[:N_img_each]:#on prend N_img_each images par categories
        
        if not img.endswith('.JPG'):
            continue
        img_array=imread(os.path.join(path,img)) 
        img_resized=resize(img_array,(256,256,3))     #  
        flat_data_arr.append(img_resized.flatten())   # On formate l'image pour numpy    
        target_arr.append(Categories.index(i))        #
    
    print(f'chargement categorie:{i} fait')
    
flat_data=np.array(flat_data_arr) #
target=np.array(target_arr)       #
df=pd.DataFrame(flat_data)        # On formate l'image pour pandas
df['Target']=target               #


x=df.iloc[:,:-1] #entrée
y=df.iloc[:,-1] #sortie 

print("Execution : ",  time.time()- t1, " s")

loading... categorie : healthy
chargement categorie:healthy fait
loading... categorie : rot
chargement categorie:rot fait
loading... categorie : rust
chargement categorie:rust fait
Execution :  77.03166007995605  s


In [ ]:
#Creation du modele avec recherche de kernel

param_grid={'C':[10000],'gamma':[0.0001],'kernel':['rbf']}#Quelles valeures à tester                      
svc=svm.SVC(probability=True)                                                   
model=GridSearchCV(svc,param_grid)                                                   

In [3]:
#Creation du modele avec kernel defini

kernel = 'rbf'
gamma = 1e-4
C = 1e5

model = svm.SVC(C = C, kernel = kernel, gamma = gamma, probability = True)

In [4]:
#On sépare les données en deux groupes: entrainement et test

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)

print('Séparation reussie')

Séparation reussie


In [5]:
#PCA
t1 = time.time()

Ncomp = 2
pca = PCA(Ncomp)
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)


print("Execution : ", -t1 + time.time(), " s")

Execution :  197.25702691078186  s


In [6]:
#Entrainement du model

t1 = time.time()

model.fit(x_train,y_train) 

print("Fin de l'entrainement")

print("Execution : ", -t1 + time.time(), " s")

Fin de l'entrainement
Execution :  11.488438129425049  s


In [ ]:
#OPTIONEL
#Stockage du model dans un fichier 

model_name = " "

with open(model_name +" "+ str(N_img_each) + ' image', 'wb') as f1:
    pickle.dump(model, f1)

In [ ]:
#OPTIONEL
#Importation d'un model

file_name = " "

with open('rbf auto 300', 'rb') as f1:
    model = pickle.load(f1)

In [7]:
#Test du model
y_pred=model.predict(x_test)

def accuracy(pred,real):
    """Calule la précision des prédictions pred par rapport à real """
    count = 0
    L = len(pred)
    for i in range(L):
        if pred[i] == real[i]:
            count += 1
    return count/L


def confusion_mat(pred,real):
    """Calcule la matrice de confusion des résultats"""
    M = np.full((3,3), 0)
    L = len(pred)
    for i in range(L):
        M[real[i]][pred[i]] += 1
        
    return M

          
print("Precision des prédiction : ", accuracy(y_pred,np.array(y_test)))
print("\n\nMatrice de confusion : ")
print(confusion_mat(y_pred,np.array(y_test)))

Precision des prédiction :  0.9813333333333333


Matrice de confusion : 
[[247   1   2]
 [  6 242   2]
 [  0   3 247]]
